In [1]:
import cv2
import os
import rasterio as rio
import numpy as np
from tqdm import tqdm
from osgeo import gdal
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from keras.models import load_model
from rasterio.plot import reshape_as_image, reshape_as_raster
import import_ipynb
from metrics import iou

importing Jupyter notebook from metrics.ipynb


In [2]:
"""laod model"""
from keras.models import load_model
dependencies = {
    'iou': iou
}
path_model = os.path.join("E:\\Tesis MPJ\\Training_lab\\Run_with SAM\\Run_SAM_hr_2\\Training_newSSUNet\\model.h5")
model = load_model(path_model, custom_objects=dependencies)

C:\Users\asus\anaconda3\envs\marzuki\lib\site-packages\keras\layers\core\lambda_layer.py:327: UserWarning: Model_1 is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [6]:
"""load image"""
imgs = []
path = "E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\"
#path = "E:\\Tesis MPJ\\SAM\\Dataset_harmonized\\Split_data\\val\\Image2\\"
valid_images = [".tif"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    imgs.append(os.path.join(path,f))
imgs

['E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_110_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_111_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_112_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_113_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_114_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_115_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_116_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_137_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\predict_image_138_sel_128.tif',
 'E:\\Tesis MPJ\\Predict_lab\\analisi

In [7]:
def read_image2(path):
    #path = path.decode()
    x = gdal.Open(path).ReadAsArray()
    x = np.rollaxis(x, 0, 3)
    #x = np.expand_dims(x,axis=-1)
    x = x/10000
    x = x.astype(np.float32)
    return x

In [5]:
for k in tqdm(imgs):
    test = read_image2(k)
    print(test.shape)
    
    t0 = rio.open(k)
    meta = t0.meta
    
    meta.update({'driver':'GTiff', 
                 'width':128, 
                 'height':128, 
                 'count':1, 
                 'dtype':'int16', 
                 'nodata':0}) 
    
    name = k.split ("\\") [- 1] .split (".") [0]
    y_pred = model.predict(np.expand_dims(test, axis=0))[0] > 0.5
    h, w, _ = test.shape
    white_line = np.ones((h, 10, 3)) * 255.0
    all_images = y_pred * 255.0
    #all_images = [
        #mask_parse(y_pred) * 255.0
    #]
    name = k.split("/")[-1].split(".") 

    """ Extracting the name and extension of the image and the mask. """ 

    image_name = name[0] 
    image_extn = name[1] 
    
    path_save = "E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\26 juli 23\\results"
    
    if not os.path.exists(path_save):
        os.makedirs(path_save)
    
    tmp_img_name = f"{image_name}_results.{image_extn}" 
    image_path = os.path.join(path_save, tmp_img_name) 
    
    image = np.rollaxis(all_images, axis = 2)
    #image = reshape_as_raster(all_images)
    #print("image shape: ", image.shape)
    with rio.open(image_path,'w',**meta) as dst: 
        dst.write(image) 
    

  0%|          | 0/323 [00:00<?, ?it/s]

(128, 128, 7)
1/1 [==============================] - 2s 2s/step


  0%|          | 1/323 [00:02<14:08,  2.64s/it]

(128, 128, 7)
1/1 [==============================] - 0s 215ms/step


  1%|          | 2/323 [00:03<07:14,  1.35s/it]

...

 99%|█████████▉| 321/323 [02:08<00:00,  2.55it/s]

(128, 128, 7)
1/1 [==============================] - 0s 160ms/step


100%|█████████▉| 322/323 [02:08<00:00,  2.54it/s]

(128, 128, 7)
1/1 [==============================] - 0s 158ms/step


100%|██████████| 323/323 [02:09<00:00,  2.50it/s]


In [8]:
# Process and save images
output_path = "E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\27 april 24\\results"
if not os.path.exists(output_path):
    os.makedirs(output_path)

for k in tqdm(imgs):
    test = read_image2(k)
    print(test.shape)
    
    t0 = rio.open(k)
    meta = t0.meta
    
    meta.update({
        'driver': 'GTiff',
        'width': 128,
        'height': 128,
        'count': 1,
        'dtype': 'int16',
        'nodata': 0
    }) 
    
    y_pred = model.predict(np.expand_dims(test, axis=0))[0] > 0.5
    all_images = y_pred * 255.0
    
    # Extract image name and extension
    name = os.path.basename(k).split(".")
    image_name = name[0]
    image_extn = name[1]
    
    # Construct output image path
    tmp_img_name = f"{image_name}_results.{image_extn}"
    image_path = os.path.join(output_path, tmp_img_name)
    
    # Roll axis for correct shape
    image = np.rollaxis(all_images, axis=2)
    
    # Write output image
    with rio.open(image_path, 'w', **meta) as dst:
        dst.write(image)
    
    #print(f"Saved result to {image_path}")

  0%|          | 0/323 [00:00<?, ?it/s]

(128, 128, 7)
1/1 [==============================] - 0s 460ms/step


  0%|          | 1/323 [00:01<05:25,  1.01s/it]

(128, 128, 7)
1/1 [==============================] - 0s 289ms/step


  1%|          | 2/323 [00:01<04:06,  1.30it/s]

...

 99%|█████████▉| 321/323 [02:13<00:00,  2.32it/s]

(128, 128, 7)
1/1 [==============================] - 0s 176ms/step


100%|█████████▉| 322/323 [02:14<00:00,  2.36it/s]

(128, 128, 7)
1/1 [==============================] - 0s 173ms/step


100%|██████████| 323/323 [02:14<00:00,  2.40it/s]


In [6]:
from rasterio.merge import merge
from rasterio.plot import show
import glob

# Define the path to the directory containing the raster files
input_directory = "E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\26 juli 23\\results"
output_file = "E:\\Tesis MPJ\\Predict_lab\\analisis temporal\\pitu sunggu\\Full_predict\\predict_26 juli 23.tif"

# Get a list of all the raster files in the directory
raster_files = glob.glob(os.path.join(input_directory, "*.tif"))

# Open all the raster files and store their references
src_files_to_mosaic = []
for fp in raster_files:
    src = rio.open(fp)
    src_files_to_mosaic.append(src)

# Merge the rasters
mosaic, out_trans = merge(src_files_to_mosaic)

# Initialize a background array with zeros
background = np.zeros_like(mosaic)

# Overlay the mosaic on the background
background[mosaic > 0] = mosaic[mosaic > 0]

# Copy the metadata from one of the files
out_meta = src.meta.copy()

# Update the metadata to reflect the number of layers and the new transform
out_meta.update({
    "driver": "GTiff",
    "height": background.shape[1],
    "width": background.shape[2],
    "transform": out_trans,
    "count": background.shape[0],
    "dtype": "uint8"  # Assuming the background and object classes are 0 and 255 respectively
})

# Write the mosaic raster with background to a new file
with rio.open(output_file, "w", **out_meta) as dest:
    dest.write(background)

# Close all the source files
for src in src_files_to_mosaic:
    src.close()

print(f"Mosaic with background saved as {output_file}")

Mosaic with background saved as E:\Tesis MPJ\Predict_lab\analisis temporal\pitu sunggu\Full_predict\predict_26 juli 23.tif
